In [1]:
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os 

In [2]:
from tensorflow.python.keras.callbacks import TensorBoard

In [3]:
from time import time

In [4]:
train_path = 'G:\\PYTHON\\Final year project\\Dataset\\train'
val_path = 'G:\\PYTHON\\Final year project\\Dataset\\val'
test_path = 'G:\\PYTHON\\Final year project\\Dataset\\test'
#test_path= test_data

In [5]:
# re-size all the images to a size VGG-16 expects.
IMAGE_SIZE = [224, 224]

# Set the batch size
BATCH_SIZE = 16  # try reducing batch size or freeze more layers if your GPU runs out of memory
NUM_EPOCHS = 5
LEARNING_RATE = 0.0001
NUM_CLASSES = 2 # We are aware of it.

In [6]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

In [7]:
# Import the images from the train dataset.
# Make sure to provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(
    directory = train_path,
    target_size = (224, 224),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

Found 6030 images belonging to 2 classes.


In [8]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [9]:
# Import the images from the test dataset.

test_set = test_datagen.flow_from_directory(
    directory = test_path,
    target_size = (224, 224),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

Found 860 images belonging to 2 classes.


In [10]:
xc = Xception(input_shape = IMAGE_SIZE + [3], weights='imagenet', include_top=False)

83697664/83683744 [==============================] - 13s 0us/step


In [12]:
for layer in xc.layers:
    layer.trainable = False

In [13]:
# our layers - you can add more if you want
x = Flatten()(xc.output)

prediction = Dense(NUM_CLASSES, activation='softmax')(x)

In [14]:
model = Model(inputs=xc.input, outputs=prediction)

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [16]:
Tensorboard = TensorBoard(log_dir="Logs\\{}".format(time()))

In [17]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [18]:
%%timeit -n1 -r1 
with tf.device('/GPU:0'):
    history = model.fit(
        training_set,
        validation_data=test_set,
        epochs=20,
        steps_per_epoch=len(training_set),
        validation_steps=len(test_set),
        callbacks=[Tensorboard]
    )

C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/20
377/377 [==============================] - 270s 683ms/step - loss: 1.3427 - accuracy: 0.8446 - val_loss: 2.4927 - val_accuracy: 0.7547
Epoch 2/20
377/377 [==============================] - 112s 296ms/step - loss: 1.2766 - accuracy: 0.8771 - val_loss: 1.8439 - val_accuracy: 0.7640
Epoch 3/20
377/377 [==============================] - 111s 295ms/step - loss: 1.3031 - accuracy: 0.8881 - val_loss: 4.4309 - val_accuracy: 0.7384
Epoch 4/20
377/377 [==============================] - 110s 290ms/step - loss: 1.0700 - accuracy: 0.9058 - val_loss: 4.8912 - val_accuracy: 0.7070
Epoch 5/20
377/377 [==============================] - 126s 333ms/step - loss: 1.0214 - accuracy: 0.9065 - val_loss: 4.8317 - val_accuracy: 0.7360
Epoch 6/20
377/377 [==============================] - 165s 437ms/step - loss: 0.8978 - accuracy: 0.9156 - val_loss: 2.1897 - val_accuracy: 0.7791
Epoch 7/20
377/377 [==============================] - 194s 514ms/step - loss: 1.3367 - accuracy: 0.9028 - val_loss: 2.6040 -

In [24]:
validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_set = validation_datagen.flow_from_directory(
    directory = val_path,
    target_size = (224, 224),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

Found 1726 images belonging to 2 classes.


In [25]:
validation_steps = 200

loss0,accuracy0 = model.evaluate(validation_set, steps = validation_steps)

print("loss: {:.2f}".format(loss0))
print("accuracy: {:.2f}".format(accuracy0))

200/200 [==============================] - 32s 159ms/step - loss: 2.8156 - accuracy: 0.8517
loss: 2.82
accuracy: 0.85


In [26]:
# Generate Validation set.
validation_set2 = validation_datagen.flow_from_directory(
    directory = val_path,
    target_size = (224, 224),
    batch_size = 1,
    shuffle=False, 
    seed=42, 
    class_mode="binary"
)

# validation_set2.reset()

Found 1726 images belonging to 2 classes.


In [27]:
# just capture the loss and accuray into val variable... unlike in pervious code to capture into loss0 and accuracy0. Just to showcase alternate way.

test = model.evaluate(test_set)

print("loss: {:.2f}".format(test[0]))
print("accuracy: {:.2f}".format(test[1]))

54/54 [==============================] - 6s 117ms/step - loss: 3.5731 - accuracy: 0.8198
loss: 3.57
accuracy: 0.82


In [28]:
model.save("G:\\PYTHON\\Final year project\\weights\\xception_weights.h5")